In [1]:
### Import the necessary libraries and packages
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import pandas as pd
import tensorflow as tf
import scipy.misc
%matplotlib inline


In [2]:
# Define a function to extract the target objects from the larger Udacity image and store them to a folder
def extract_images(data, filepath):
    for ii, img in enumerate(data):
         # Read in each one by one
        file = img[0]
        image = mpimg.imread(file)
        # Isolate the region in the image that contains the object
        xmin, ymin, xmax, ymax = img[1], img[2], img[3], img[4]
        image = image[ymin:ymax + 1, xmin:xmax + 1]
        image = cv2.resize(image, (64, 64))
        filler = '00000'
        mpimg.imsave(filepath + 'image_' + filler[:len(filler) - len(str(ii))] + str(ii) + '.jpg', image)
        

In [3]:
def convert_color(img, conv=None):
    image = np.copy(img)
    if conv != 'RGB':
        transform = "cv2.cvtColor(image, cv2.COLOR_" + conv + ")"
        features = eval(transform)
    else:
        features = image
    
    return features
    
# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32), channel='all'):

    if channel == 'all':
        channel = np.arange(3)
    
    features = []
    for ch in channel:
        color = cv2.resize(img[:, :, ch], size).ravel()
        features.append(color)
    spatial_features = np.concatenate(features)
            
    return spatial_features

# Define a function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256), channel='all'):
    # Compute the histogram of the color channels separately
    if channel == 'all':
        channel = np.arange(3)
        
    features = []
    hist_features = []
    for ch in channel:
        channel_hist = np.histogram(img[:, :, ch], bins=nbins, range=bins_range)
        features.append(channel_hist[0])
    hist_features = np.concatenate(features)
        
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True, trans_sqrt=True, block_norm='L1'):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), block_norm=block_norm,
                                  transform_sqrt=trans_sqrt, visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), block_norm=block_norm,
                       transform_sqrt=trans_sqrt, visualise=vis, feature_vector=feature_vec)
        return features
    

In [4]:
# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(data, img_format='png', spatial_dict=None, hist_dict=None, hog_dict=None):
    
    # Create a list to append feature vectors to
    features = []
    
    # Iterate through the list of images
    for file in data:
        single_img_features = []
        # Read in each one by one
        image = mpimg.imread(file) 
        if ((img_format == 'jpg') | (file[-4:] == '.jpg')):
            image = image.astype(np.float32) / 255
        image = cv2.resize(image, (64, 64))
        
        if spatial_dict is not None:
            cspace = spatial_dict['conv']
            spatial_size = spatial_dict['size']
            channels = spatial_dict['channels']
            image_conv = convert_color(image, cspace)
            spatial_features = bin_spatial(image_conv, size=(spatial_size, spatial_size), channel=channels)
            single_img_features.append(spatial_features)
            
        if hist_dict is not None:
            cspace = hist_dict['conv']
            hist_bins = hist_dict['nbins']
            bin_range = hist_dict['bin_range']
            channels = hist_dict['channels']
            image_conv = convert_color(image, cspace)
            hist_features = color_hist(image_conv, nbins=hist_bins, bins_range=(0, 256), channel=channels)
            single_img_features.append(hist_features)

        if hog_dict is not None:
            cspace = hog_dict['conv']
            orient = hog_dict['orient']
            pix_per_cell = hog_dict['pix_per_cell']
            cell_per_block = hog_dict['cell_per_block']
            channels = hog_dict['channels']       
            trans_sqrt = hog_dict['trans_sqrt']
            block_norm = hog_dict['block_norm']
            image_conv = convert_color(image, cspace)  
            hog_features = []
            if channels == 'all':
                channels = np.arange(3)
            
            for ch in channels:
                hog_features.append(get_hog_features(image_conv[:,:,ch], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True, trans_sqrt=trans_sqrt, block_norm=block_norm))
            hog_features = np.ravel(hog_features)                      
            single_img_features.append(hog_features)
            
        features.append(np.concatenate(single_img_features))

    # Return list of feature vectors
    return features


In [5]:
import pickle

# Save a dictionary into a pickle file
def save_to_pickle(data, key_name, file_name):
    if len(file_name) > 1:
        for d, k, f in zip(data, key_name, file_name):
            pickle_data = {k: d}
            pickle.dump(pickle_data, open(f + '.p', "wb"))
    else:
        pickle_data = {}
        for d, k in zip(data, key_name):
            pickle_data[k] = d
        print(pickle_data)
        pickle.dump(pickle_data, open(file_name[0] + '.p', "wb"))
    
    return None
        

In [6]:
def make_training_files(len_non_cars=None, GTI=False, KITTI=False, udacity=False):
    
    car_gti = []
    kitti_cars = []
    car_gti_far = []
    car_gti_left = []
    car_gti_midclose = []
    car_gti_right = []
    
    if GTI:
        # Collect the filenames for all car images in the dataset
        car_gti_far = glob.glob('../data/vehicles/GTI_Far/*png')
        car_gti_left = glob.glob('../data/vehicles/GTI_Left/*png')
        car_gti_midclose = glob.glob('../data/vehicles/GTI_MiddleClose/*png')
        car_gti_right = glob.glob('../data/vehicles/GTI_Right/*png')
        car_gti = car_gti_far + car_gti_left + car_gti_midclose + car_gti_right
    if KITTI:
        kitti_cars = glob.glob('../data/vehicles/KITTI_extracted/*png')

    # Collect the filenames for all non-car images in the dataset
    non_car_gti = glob.glob('../data/non_vehicles/GTI/*png')
    extras_non_cars = glob.glob('../data/non_vehicles/Extras/*png')
    
    car_files = np.array(car_gti + kitti_cars)
    non_car_files = np.array(non_car_gti + extras_non_cars)
    non_car_files = shuffle(non_car_files)
    if len_non_cars: 
        n_sample = len_non_cars
    else:
        n_sample = len(non_car_files)
    
    udacity_car_files = np.array([])
    if udacity:
        udacity_df = pd.read_csv('../data/labels.csv')
        udacity_cars_df = udacity_df.loc[(udacity_df.label == 'car') & (udacity_df.occluded == 0)]

        udacity_car_idx = udacity_cars_df.index.tolist()
        udacity_car_files = np.array(glob.glob('../data/udacity_cropped/*jpg'))
        udacity_car_files = udacity_car_files[udacity_car_idx]

    all_img_files = np.hstack((non_car_files[:n_sample], car_files, udacity_car_files))
    img_type_labels = np.hstack((np.zeros(n_sample), np.ones(len(car_gti_far)), np.ones(len(car_gti_left)) * 2, 
                                     np.ones(len(car_gti_midclose)) * 3, np.ones(len(car_gti_right)) * 4, 
                                     np.ones(len(kitti_cars)) * 5, np.ones(len(udacity_car_files)) * 6))

    print('No. of GTI car files: {}'.format(len(car_gti)))
    print('No. of KITTI car files: {}'.format(len(kitti_cars)))
    print('No. of Udacity car files: {}'.format(len(udacity_car_files)))
    print('Total number of car files: {}'.format(len(car_files) + len(udacity_car_files)))
    print('Tota number of non-car files: {}'.format(n_sample))
    print('Total number of images: {}'.format(all_img_files.shape))
    print('Total number of labels: {}'.format(img_type_labels.shape))

    train_img_files = np.copy(all_img_files)
    train_labels_copy = np.copy(img_type_labels)
    
    return train_img_files, train_labels_copy


In [ ]:
train_img_files, train_labels_copy = make_training_files(KITTI=True, len_non_cars=7000)

img_format = 'png'
## Here is a list of possible color spaces to use
'''cv2.COLOR_RGB2HSV
   cv2.COLOR_RGB2LUV
   cv2.COLOR_RGB2HLS
   cv2.COLOR_RGB2YUV
   cv2.COLOR_RGB2YCrCb'''

spatial_dict = {'conv': 'RGB2YCrCb', 'size': 32, 'channels': 'all'}
hist_dict = {'conv': 'RGB2YCrCb', 'nbins': 32, 'bin_range': (0, 256), 'channels': 'all'}
# spatial_dict = None
# hist_dict = None
hog_dict = {'conv': 'RGB2YCrCb', 'orient': 9, 'pix_per_cell': 8, 'cell_per_block': 2, 'channels': 'all',
            'trans_sqrt': True, 'block_norm': 'L1'}

X_train_features = extract_features(train_img_files, spatial_dict=spatial_dict, hist_dict=hist_dict, hog_dict=hog_dict)
file_end = 'ycrcb_kitti_only'

X_train_features = np.array(X_train_features).astype(np.float32)
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X_train_features)
# Apply the scaler to X
scaled_X = X_scaler.transform(X_train_features)
print('X train features shape is {}'.format(X_train_features.shape))
print('Scaled X features is {}'.format(scaled_X.shape))

save_to_pickle([scaled_X, train_labels_copy, X_scaler], ['features', 'labels', 'scaler'], 
               ['../pickle/scaled_X_' + file_end])
save_to_pickle([spatial_dict, hist_dict, hog_dict], ['spatial', 'hist', 'hog'], 
               ['../pickle/dict_' + file_end])

rand_state = np.random.randint(0, 100)
# Shuffle and split the data into a training and test set
X_train, X_validate, y_train, y_validate = train_test_split(scaled_X, train_labels_copy, test_size=0.2, 
                                                            stratify=train_labels_copy, random_state=rand_state)
print('X_train shape is {}'.format(X_train.shape))
print('y_train shape is {}'.format(y_train.shape))
print('X_validate shape is {}'.format(X_validate.shape))
print('y_validate shape is {}'.format(y_validate.shape))

print(y_train[:20])
y_train[y_train != 0] = 1.
y_validate[y_validate != 0] = 1.
print(y_train[:20])


In [ ]:
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
print('Test Accuracy of SVC = ', round(svc.score(X_validate, y_validate), 4))
save_to_pickle([svc, hog_dict['trans_sqrt'], hog_dict['block_norm']], ['svc', 'trans_sqrt', 'block_norm'], 
               ['../pickle/svc_pickle_' + file_end])


In [7]:
# Define a single function that can extract features using hog sub-sampling 
# and make predictions
def find_cars(img, img_format, xstart, ystart, ystop, scale, X_scaler, svc, hog_dict, 
              spatial_dict=None, hist_dict=None):
    
    if spatial_dict is not None:
        sp_cspace = spatial_dict['conv']
        spatial_size = spatial_dict['size']
        sp_ch = spatial_dict['channels']

    if hist_dict is not None:
        hs_cspace = hist_dict['conv']
        hist_bins = hist_dict['nbins']
        bin_range = hist_dict['bin_range']
        hi_ch = hist_dict['channels']
            
    # Extract the Hog parameters
    hg_cspace = hog_dict['conv']
    orient = hog_dict['orient']
    pix_per_cell = hog_dict['pix_per_cell']
    cell_per_block = hog_dict['cell_per_block']
    hg_ch = hog_dict['channels']       
    trans_sqrt = hog_dict['trans_sqrt']
    block_norm = hog_dict['block_norm']
    
    if hg_ch == 'all':
        hg_ch = np.arange(3)
    
    draw_img = np.copy(img)
    if img_format == 'jpg':
        img = img.astype(np.float32) / 255
     
    # Compute individual channel HOG features for the entire image
    img_tosearch = img[ystart:ystop,xstart:,:]
    ctrans_tosearch = convert_color(img_tosearch, conv=hg_cspace)
    bboxes = [] # Store the coordinates of the bounded boxes

    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1] / scale), 
                                     np.int(imshape[0] / scale)))
    chs = []
    for ch in hg_ch:
        chs.append(ctrans_tosearch[:,:,ch])

    # Define blocks and steps
    nxblocks = (chs[0].shape[1] // pix_per_cell) - cell_per_block + 1
    nyblocks = (chs[0].shape[0] // pix_per_cell) - cell_per_block + 1 
    nfeat_per_block = orient*cell_per_block**2

    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step + 1
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step + 1

    hogs = []
    for ch in chs:
        hog_feature = get_hog_features(ch, orient, pix_per_cell, cell_per_block, 
                                feature_vec=False, trans_sqrt=trans_sqrt, block_norm=block_norm)
        hogs.append(hog_feature)
#     print("hog_feature shape: {}".format(hogs[0].shape))
    
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step

            # Extract HOG for this patch
            hog_list = []
            for hg in hogs:    
                hog_vect = hg[ypos:ypos+nblocks_per_window, 
                                 xpos:xpos+nblocks_per_window].ravel()
                hog_list.append(hog_vect)   
#             print("individual hog shape: {}".format(hogs[0][ypos:ypos+nblocks_per_window, 
#                                  xpos:xpos+nblocks_per_window].shape))
            hog_features = np.hstack(hog_list)
        
            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell
            
#             print('original ctrans shape: {}'.format(imshape))
#             print('resized ctrans shape: {}'.format(ctrans_tosearch.shape))
#             print('xleft, xleft + window: {}, {}'.format(xleft, xleft + window))
#             print('window ctrans shape: {}'.format(ctrans_tosearch[ytop:ytop+window, 
#                                                    xleft:xleft+window].shape))
#             print('\n')
            
           # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, 
                                xleft:xleft+window], (64,64))

            # Get color features
            spatial_features = np.array([])
            hist_features = np.array([])
            if spatial_dict is not None:
                spatial_features = bin_spatial(subimg, size=(spatial_size, spatial_size), channel=sp_ch)
            if hist_dict is not None:
                hist_features = color_hist(subimg, nbins=hist_bins, bins_range=(0, 256), channel=hi_ch)

            # Scale features and make a prediction
            features = np.hstack((spatial_features, hist_features, hog_features))
            test_features = X_scaler.transform(features.reshape(1, -1))       
            test_prediction = svc.predict(test_features)
            
            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                cv2.rectangle(draw_img,(xbox_left+xstart, ytop_draw+ystart),
                (xbox_left+xstart+win_draw,ytop_draw+win_draw+ystart),(0,0,255),6) 
                bboxes.append(((xbox_left + xstart, ytop_draw + ystart), 
                               (xbox_left + xstart + win_draw, ytop_draw + win_draw + ystart)))
                
    return draw_img, bboxes


In [8]:
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap

def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap < threshold] = 0
    # Return thresholded map
    return heatmap


In [9]:
from scipy.ndimage.measurements import label

def draw_labeled_bboxes(img, labels):
    out_bboxes = {} # A dict of all the bounded boxes for each car found in this mage
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        out_bboxes[car_number] = bbox
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 3)
        
    # Return the image
    return img, out_bboxes


In [10]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML


In [11]:
# Define a class to keep track of data among frames
class Frame():
    def __init__(self):
        self.count = 1 # Tracks the number of frames that have passed
        self.bboxes = [] # Each entry in this list is the detection boxes for one frame; used for
                         # accumulating detection boxes for heat map threshold
        self.samples = 6 # How many frames should we collect boxes for
        self.centroids = [] # A list of the centroids of each bounding box around a car
        self.image = None
        self.detect_boxes = None # An flattened version of self.bboxes
        self.car_boxes = None # A list of the bounded box surrounding an entire car
        

In [17]:
# This is the function that processes each frame of the video to find cars and apply a box around them
def process_image(image):
    
#     if frame.image is None:
    frame.image = np.copy(image)
    
    far_bboxes = []
    mid_bboxes = []
    close_bboxes = []
#     # Search for cars in the distance
    xstart = 0
#     ystart = 400
#     ystop = 450
#     scale = 0.5
# #     print('far')
#     _, far_bboxes = find_cars(frame.image, img_format, xstart, ystart, ystop, scale, X_scaler, 
#                               svc, hog_dict, spatial_dict, hist_dict)
    # Search for cars in the middle
    ystart = 400
    ystop = 550
    scale = 1
#     print('mid')
    _, mid_bboxes = find_cars(frame.image, img_format, xstart, ystart, ystop, scale, X_scaler, 
                              svc, hog_dict, spatial_dict, hist_dict)
    # Search for close cars
    ystart = 400
    ystop = 650
    scale = 1.5
#     print('close')
#     _, close_bboxes = find_cars(frame.image, img_format, xstart, ystart, ystop, scale, X_scaler, 
#                                 svc, hog_dict, spatial_dict, hist_dict)
    total_bboxes = far_bboxes + mid_bboxes + close_bboxes
    frame.bboxes.append(total_bboxes)
           
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame.image, str(len(frame.bboxes)), (650, 150), font, 1, (200,255,155), 2, cv2.LINE_AA)

    if len(frame.bboxes) >= frame.samples:
        frame.detect_boxes = [item for sublist in frame.bboxes for item in sublist]
        
        for box in frame.detect_boxes:
            cv2.rectangle(frame.image, box[0], box[1], (255, 255, 0), 2)

        # Accumulate bounded boxes across several frames and then apply heat-map with threshold
        heat = np.zeros_like(image[:,:,0]).astype(np.float)
        heat = add_heat(heat, frame.detect_boxes)
        heat_map = apply_threshold(heat, 5)
        labels = label(heat_map)
        frame.image, frame.car_boxes = draw_labeled_bboxes(frame.image, labels)
        # Reset the store bboxes
        frame.bboxes = []
    else:
        if frame.detect_boxes is not None:
            for box in frame.detect_boxes:
                cv2.rectangle(image, box[0], box[1], (255, 255, 0), 2)
        
        if frame.car_boxes:
            for car, box in frame.car_boxes.items():
                cv2.rectangle(frame.image, box[0], box[1], (0, 0, 255), 2)
                box_center = ((box[1][0] + box[0][0]) // 2, (box[1][1] + box[0][1]) // 2)
                frame.centroids.append(box_center)
        
#     if frame.centroids:
#         for center in frame.centroids:
#             cv2.rectangle(frame.image, (center[0] - 2, center[1] - 2), (center[0] + 2, center[1] + 2), (255, 0, 0), 2)
    
    return frame.image


In [13]:
file_end = 'ycrcb'
dict_pickle = pickle.load( open("../pickle/scaled_X_" + file_end + ".p", "rb") )
X_scaler = dict_pickle['scaler']

dict_pickle = pickle.load( open("../pickle/dict_" + file_end + ".p", "rb") )
spatial_dict = dict_pickle['spatial']
hist_dict = dict_pickle['hist']
hog_dict = dict_pickle['hog']

dict_pickle = pickle.load( open("../pickle/svc_pickle_" + file_end + ".p", "rb" ) )
svc = dict_pickle['svc']

In [18]:
img_format = 'jpg'
frame = Frame()
def count_frames(image):
    result = np.copy(image)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(result, str(frame.count), (600, 150), font, 1, (200,255,155), 2, cv2.LINE_AA)
    frame.count += 1
    return result

clip1 = VideoFileClip("../project_video.mp4", audio=False)
road_clip = clip1.fl_image(process_image)
%time road_clip.write_videofile("../project_output.mp4", audio=False, verbose=0)

100%|█████████▉| 1260/1261 [12:00<00:00,  1.75it/s]


CPU times: user 11min 44s, sys: 18.9 s, total: 12min 3s
Wall time: 12min
